# Quick MNIST

*by Jeremy Howard, fast.ai.*

In this notebook we'll see how easy it is to use the simple functions we created along with `torch.nn` and friends to train MNIST. We're going to create the same convolution net that we created at the end of the previous notebook, but this time we'll use the functions we've already written, and we'll skip the explanatory text so you can see just the final code.

You can use this same notebook to train other neural nets on other datasets with minimal changes.

## Modulo `mnist_sample.py`

In [27]:
import numpy as np
import torch
import torch.nn.functional as F
from torch import nn,optim,tensor
from torch.utils.data import TensorDataset, DataLoader
import pickle, gzip



def fit(epochs, model, loss_func, opt, train_dl, valid_dl):
    for epoch in range(epochs):
        # Entrenamiento
        model.train()
        for xb,yb in train_dl: 
            pb = model(xb)
            loss = loss_func(pb,yb)  
            loss.backward()
            opt.step()
            opt.zero_grad()
        # Validación
        model.eval()
        losses = list()
        nums = list()
        with torch.no_grad():
            for xb,yb in valid_dl:
                pb = model(xb)
                loss  = loss_func(pb,yb)
                num = len(xb)
                losses.append(loss.item())
                nums.append(num)
        val_loss = np.sum(np.multiply(losses,nums)) / np.sum(nums)
        print(epoch, val_loss)




class WrappedDataLoader():
    def __init__(self, dl, func):
        self.dl = dl
        self.func = func

    def __len__(self): return len(self.dl)

    def __iter__(self):
        batches = iter(self.dl)
        for b in batches: yield(self.func(*b))



## Prueba del módulo

In [28]:


with gzip.open('data/mnist.pkl.gz', 'rb') as f:
    ((train_x, train_y), (valid_x, valid_y), _) = pickle.load(f, encoding='latin-1')

In [29]:
bs=64
lr=0.1
epochs=4

dev = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [30]:
def preprocess(x,y): return x.view(-1,1,28,28).to(dev),y.to(dev)

def get_dataloader(x,y,bs,shuffle):
    ds = TensorDataset(*map(tensor, (x,y)))
    dl = DataLoader(ds, batch_size=bs, shuffle=shuffle)
    for xb, yb in dl:
        xp = xb.view(-1,1,28,28).to(dev)
        yp = yb.to(dev)
    return xp,yp

In [31]:
train_dl = get_dataloader(train_x, train_y, bs,   shuffle=False)
valid_dl = get_dataloader(valid_x, valid_y, bs*2, shuffle=True )

In [32]:
# Define a generic custom layer
class Custom_layer(nn.Module):
    def __init__(self, func):
        super().__init__()
        self.func = func
    def forward(self, x):
        return self.func(x)

In [33]:
model = nn.Sequential(
    nn.Conv2d(1,  16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 16, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.Conv2d(16, 10, kernel_size=3, stride=2, padding=1), nn.ReLU(),
    nn.AdaptiveAvgPool2d(1),
    Custom_layer(lambda x: x.view(x.size(0),-1))
).to(dev)
loss_func = F.cross_entropy
opt = optim.SGD(model.parameters(), lr=lr, momentum=0.9)

In [35]:
fit(epochs, model, loss_func, opt, train_dl, valid_dl)

ValueError: too many values to unpack (expected 2)